In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%pylab inline
from google.colab import files
uploaded = files.upload()

Populating the interactive namespace from numpy and matplotlib


Saving telecom.csv to telecom.csv


In [2]:
#https://stackoverflow.com/questions/48152674/how-to-check-if-pytorch-is-using-the-gpu
seed = 7
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
torch.cuda.is_available()

False

In [3]:
import io 
df = pd.read_csv(io.BytesIO(uploaded['telecom.csv']))
ids = df[['customerID']]
df.drop(columns=['customerID'], inplace=True)
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [4]:
l = []
for j, i in enumerate(df['TotalCharges']):
    try:
        float(i)
    except ValueError:
        print(j, '_________', i)
        l.append(j)
df.drop(l, inplace = True ) 
df['TotalCharges'] = df['TotalCharges'].astype(float)
df = df.reset_index( drop = True )

488 _________  
753 _________  
936 _________  
1082 _________  
1340 _________  
3331 _________  
3826 _________  
4380 _________  
5218 _________  
6670 _________  
6754 _________  


In [5]:
binary = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
categorical = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']
numerical = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']
df = pd.get_dummies(df, columns=binary, drop_first=True)
df = pd.get_dummies(df, columns=categorical)
# Creating the time and event columns
time_column = 'tenure'
event_column = 'Churn_Yes'
 
# Extracting the features
features = np.setdiff1d(df.columns, [time_column, event_column] ).tolist()

In [6]:
from sklearn.model_selection import train_test_split

index_train, index_test = train_test_split(range(df.shape[0]), test_size=0.3, random_state=0, stratify=df['Churn_Yes'])

tmp_data_train = df.loc[index_train].reset_index( drop = True )
data_test  = df.loc[index_test].reset_index( drop = True )

index_train, index_valid = train_test_split(tmp_data_train.index, test_size=0.2, random_state=0, stratify=tmp_data_train['Churn_Yes'])
data_train = tmp_data_train.loc[index_train].reset_index( drop = True )
data_valid = tmp_data_train.loc[index_valid].reset_index( drop = True )

 
# Creating the X, T and E inputs
X_train, X_valid, X_test = data_train[features], data_valid[features], data_test[features]
T_train, T_valid, T_test = data_train[time_column], data_valid[time_column], data_test[time_column]
E_train, E_valid, E_test = data_train[event_column], data_valid[event_column], data_test[event_column]
 
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(X_train[['MonthlyCharges', 'TotalCharges']])
scaled_valid = scaler.fit_transform(X_valid[['MonthlyCharges', 'TotalCharges']])
scaled_test = scaler.transform(X_test[['MonthlyCharges', 'TotalCharges']])
X_train[['MonthlyCharges', 'TotalCharges']], X_valid[['MonthlyCharges', 'TotalCharges']], X_test[['MonthlyCharges', 'TotalCharges']] = scaled_train, scaled_valid, scaled_test

X_train.shape, X_valid.shape, X_test.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


((3937, 39), (985, 39), (2110, 39))

In [ ]:
class NNModel(nn.Module):
    def __init__(self, input_shape, units=None, factors=None, dropout=None, slops=None):
        super().__init__()
        self.input_shape = input_shape
        self.units = units
        self.factors = factors
        self.network = nn.ModuleList()
        if self.factors:
            self.units = np.round(self.input_shape * np.asarray(self.factors)).astype(int)
        if self.units is not None:
            self.dropout = np.zeros_like(self.units) if dropout is None else dropout
            self.slops = np.full(len(self.units), 1) if slops is None else slops
            for i, j, k in zip(self.units, self.dropout, self.slops):
                if i >= 1:
                    block = self.__build_block__(input_shape, i, p=j, slop=k)
                    self.network.extend(block)
                    input_shape = i
        self.output_shape = input_shape
        self.reset_parameters()
    
    def __build_block__(self, input_shape, units, p, slop, norm=False):
        block = []
        block.append(nn.Linear(input_shape, units, bias=not norm))
        if norm:
            block.append(nn.BatchNorm1d(units))
        block.append(nn.ELU(slop))
        if p > 0:
            block.append(nn.Dropout(p))
        return block
 
    def forward(self, x):
        for layer in self.network:
          tmp = layer(x)
          x = tmp
        return x
 
    def reset_parameters(self):
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)

class DeepHit:
    def __init__(self, model, num_time_bins):
        self.model = model
        self.losses = {'Epoch': [], 'Train': [], 'Test': []}
        self.num_time_bins = num_time_bins
        self.times = None

    def compute_mask1(self, time, label, num_Event, num_Category):
    '''
        mask1 is required to get the log-likelihood loss
        mask1 size is [N, num_Event, num_Category]
            if not censored : one element = 1 (0 elsewhere)
            if censored     : fill elements with 1 after the censoring time (for all events)
    '''
    mask = np.zeros([np.shape(time)[0], num_Event, num_Category]) # for the first loss function
    print(mask.shape)
    for i in range(np.shape(time)[0]):
        if label[i,0] != 0:  #not censored
            mask[i,int(label[i,0]-1),int(time[i,0])] = 1
        else: #label[i,2]==0: censored
            mask[i,:,int(time[i,0]+1):] =  1 #fill 1 until from the censoring time (to get 1 - \sum F)
    return mask


    def compute_mask2(time, meas_time, num_Category):
        '''
            mask2 is required calculate the ranking loss (for pair-wise comparision)
            mask2 size is [N, num_Category].
            - For longitudinal measurements:
                1's from the last measurement to the event time (exclusive and inclusive, respectively)
                denom is not needed since comparing is done over the same denom
            - For single measurement:
                1's from start to the event time(inclusive)
        '''
        mask = np.zeros([np.shape(time)[0], num_Category]) # for the first loss function
        if np.shape(meas_time):  #lonogitudinal measurements
            for i in range(np.shape(time)[0]):
                t1 = int(meas_time[i, 0]) # last measurement time
                t2 = int(time[i, 0]) # censoring/event time
                mask[i,(t1+1):(t2+1)] = 1  #this excludes the last measurement time and includes the event time
        else:                    #single measurement
            for i in range(np.shape(time)[0]):
                t = int(time[i, 0]) # censoring/event time
                mask[i,:(t+1)] = 1  #this excludes the last measurement time and includes the event time
        return mask

    def log_likelihood_loss(self, mask1, out):
        tmp1 = torch.log(torch.sum(torch.sum(mask1 * out, 2), 1, keepdim=True))
        return torch.mean(tmp1)

In [10]:
import tensorflow as tf
x = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])
x

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[1, 1, 1],
        [2, 2, 2]],

       [[3, 3, 3],
        [4, 4, 4]],

       [[5, 5, 5],
        [6, 6, 6]]], dtype=int32)>

In [12]:
tf.slice(x, [0, 1, 0], [-1, 1, -1])

<tf.Tensor: shape=(3, 1, 3), dtype=int32, numpy=
array([[[2, 2, 2]],

       [[4, 4, 4]],

       [[6, 6, 6]]], dtype=int32)>

In [13]:
if np.shape(-1):
  print('true')
else:
  print('false')

false
